In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
from scipy import stats
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output

# Any results you write to the current directory are saved as output.

### Target Encoding with smoothing

`min_samples_leaf` は、あるカテゴリ値に対して、事前確率（全体平均）とそのカテゴリのターゲット平均が同じ重みを持つようになるしきい値を定義します。

このしきい値よりも下では事前確率の方が重要になり、上ではカテゴリごとの平均値の方が重要になります。

カテゴリの出現回数に対する重みの変化の仕方は、`smoothing` パラメータによって制御されます。



カテゴリごとの出現回数 $\text{count}$ を使って、スムージング係数 $\text{smoothing}$ を次のように計算します。

$$
\text{smoothing} = \frac{1}{1 + \exp\left(-\frac{\text{count} - \text{min\_samples\_leaf}}{\text{smoothing}}\right)}
$$

- $\text{count}$ ：カテゴリの出現回数
- $\text{min\_samples\_leaf}$：最小サンプル数（小さいカテゴリに対して過信しすぎないための基準）
- $\text{smoothing}$（引数）：スムージングの強さを調整するパラメータ

この式は、ロジスティック関数（シグモイド関数）に似た形をしており、  
カテゴリの出現回数が増えるとスムージング係数が 1 に近づき、  
出現回数が少ないとスムージング係数が 0 に近づきます。


In [11]:
def add_noise(series,noise_level=0.01):
    return series * (1 + noise_level * np.random.randn(len(series)))

In [12]:
def target_encode(trn_series=None, 
                  tst_series=None, 
                  target=None, 
                  min_samples_leaf=1, 
                  smoothing=1,
                  noise_level=0):
    """
    trn_series:訓練用のカテゴリ変数(pandas Series)
    tst_series:テスト用のカテゴリ変数(pandas Series)
    target:目的変数(pandas Series)
    min_samples_leaf:小さいカテゴリの平均を信用しすぎないためのパラメータ
    smoothing:スムージング強度（滑らかにする）
    noise_level:後で少しノイズを加える割合（オーバーフィット防止）
    """ 
    # 訓練データとターゲットの長さが一致しているか確認
    assert len(trn_series) == len(target)
    # 訓練とテストでカラム名が一致しているか確認
    assert trn_series.name == tst_series.name
    
    temp = pd.concat([trn_series, target], axis=1)
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # priorは全体のターゲット平均。それと各カテゴリの平均をスムージングで混ぜる。
    # それと各カテゴリの平均をスムージングで混ぜる。
    #「東京の生存率はサンプル数少ないから、全体平均もちょっと混ぜるね」みたいな調整。
    prior = target.mean()
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    # 最終的に使うのはスムージングされた平均だけ。
    averages.drop(["mean", "count"], axis=1, inplace=True)

    ft_trn_series = pd.merge(
        # SeriesをDataFrameに変換
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)

    ft_trn_series.index = trn_series.index 

    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)

    ft_tst_series.index = tst_series.index

    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

### Testing with ps_car_11_cat

In [13]:
# reading data
trn_df = pd.read_csv("data/train.csv", index_col=0)
sub_df = pd.read_csv("data/test.csv", index_col=0)

# Target encode ps_car_11_cat
trn, sub = target_encode(trn_df["ps_car_11_cat"], 
                         sub_df["ps_car_11_cat"], 
                         target=trn_df.target, 
                         min_samples_leaf=100,
                         smoothing=10,
                         noise_level=0.01)
trn.head(10)

id
7     0.038848
9     0.023906
13    0.031488
16    0.044360
17    0.026202
19    0.045251
20    0.022511
22    0.030943
26    0.034956
28    0.044711
Name: ps_car_11_cat_mean, dtype: float64

### Scatter plot of category values vs taret encoding

### Check AUC metric improvement after noisy encoding over 5 folds

In [14]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import numpy as np

# StratifiedKFoldで分割設定
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

# '_cat'を含むカテゴリ特徴量だけリストアップ
f_cats = [f for f in trn_df.columns if "_cat" in f]

# ヘッダー出力
print("%20s   %20s | %20s" % ("", "Raw Categories", "Encoded Categories"))

# 各カテゴリ特徴量に対して
for f in f_cats:
    print("%-20s : " % f, end="")
    
    e_scores = []  # エンコード後スコア格納
    f_scores = []  # エンコード前スコア格納
    
    # 5-fold cross validation
    for trn_idx, val_idx in folds.split(trn_df.values, trn_df.target.values):
        
        # ❗ここを正しく修正（val_idxを使う！）
        trn_f, trn_tgt = trn_df[f].iloc[trn_idx], trn_df.target.iloc[trn_idx]
        val_f, val_tgt = trn_df[f].iloc[val_idx], trn_df.target.iloc[val_idx]

        # target encodingを学習用と検証用に適用
        trn_tf, val_tf = target_encode(
            trn_series=trn_f,
            tst_series=val_f,
            target=trn_tgt,
            min_samples_leaf=100,
            smoothing=20,
            noise_level=0.01
        )

        # 生データのROC-AUC（値が小さい場合は1-AUCを取る）
        raw_auc = roc_auc_score(val_tgt, val_f)
        f_scores.append(max(raw_auc, 1 - raw_auc))
        
        # エンコード後データのROC-AUC
        e_scores.append(roc_auc_score(val_tgt, val_tf))
    
    # Fold平均と標準偏差を出力
    print(" %.6f + %.6f | %6f + %.6f" 
          % (np.mean(f_scores), np.std(f_scores), np.mean(e_scores), np.std(e_scores)))


                             Raw Categories |   Encoded Categories
ps_ind_02_cat        :  0.506204 + 0.003512 | 0.510449 + 0.004402
ps_ind_04_cat        :  0.512617 + 0.003800 | 0.513788 + 0.006141
ps_ind_05_cat        :  0.520249 + 0.003626 | 0.534299 + 0.003785
ps_car_01_cat        :  0.528913 + 0.003454 | 0.551347 + 0.003333
ps_car_02_cat        :  0.531614 + 0.002454 | 0.529862 + 0.002166
ps_car_03_cat        :  0.539650 + 0.002949 | 0.540559 + 0.004684
ps_car_04_cat        :  0.536473 + 0.001356 | 0.536138 + 0.002215
ps_car_05_cat        :  0.530585 + 0.005116 | 0.530724 + 0.006458
ps_car_06_cat        :  0.515693 + 0.002867 | 0.541883 + 0.004697
ps_car_07_cat        :  0.522624 + 0.000995 | 0.522380 + 0.002023
ps_car_08_cat        :  0.520287 + 0.003198 | 0.517903 + 0.005479
ps_car_09_cat        :  0.504890 + 0.003429 | 0.524150 + 0.003633
ps_car_10_cat        :  0.500385 + 0.000405 | 0.498746 + 0.003641
ps_car_11_cat        :  0.512416 + 0.004256 | 0.568516 + 0.003962
